In [0]:
import fastai; fastai.__version__

Out[1]: '1.0.61'

In [0]:
import pandas; pandas.__version__

Out[2]: '1.0.0'

**image_feature_extraction**

This notebook extracts image features of the trained CNN. 
The steps to to so are as follows: 

1) specify the global parameters: 

2) Set up connections to VM and drive

3) Load the data and trained network

4) extract the features

# 1) specify global parameters

These global parameters create variablenames of the input files used throughout the code.


Only year and country need to be changed. 

there are multiple relevant year variables

survey year is the year specified in the ground truth. The "CENI" file contains the available years

The day-time sattelite images were taken in the year >>daytime_year<<. 

The >>survey_year<< 2015 uses images taken from october 2015 to october 2016. The variable >>daytime_year<< is 2016 in those cases. See the handbook for details. 
      
nl_year is the year of the nightlight data specified in the CENI file

In [0]:
country = "COL" 
survey_year = "2016"       # alternatives: "PHL"; "NER"
daytime_year = survey_year
nl_year = survey_year
prediction_year = survey_year

if (int(daytime_year) < 2015):
  day_sat = "LS"         # alternatives: "ST"
if (int(daytime_year) >= 2015):
  day_sat = "ST"


architecture = "RES34" # alternatives: "RES50"
nl_sat = "VI"          # alternatives: "DM"
img_res_dict = {"ST" : "384", "LS":"256"} # daytime image resolution handled automatically # old spec: img_res = "384"   
img_res = img_res_dict[day_sat]
img_size = "3840"      # alternatives: "2560"
number_bands = "03"    # alternatives: "04"; "12"
cardinality = "ALL"    # alternatives: "RUR"; "URB"
sd_outliers = "04"     # alternatives: "04"; "08"; "99"(no outlier exclusions)
layer_drops = 2        # how much of the network are we cutting away, NOTE: this does not correspond to single layers but smaller components (Linear weights, RELU and others)
use_pop = "usepopT"    # do we only average images with non0 populations for ridge? usepopT: filter images; usepopF: use all images
validation_size_percent = 10
validation_size = "TST" + str(validation_size_percent)  # size of the validation set, that is not used for either the CNN or Ridge


# table containing the long lat coordinates for our images, nightlights etc
# CENTroid
CENT = "_".join( ["CNN", "CENT", country, img_size, cardinality] ) + ".csv"

# table containing the long lat coordinates and corresponding nightlights etc
# NIGHttime data
NIGH = "_".join( ["CNN", "NIGH", country, nl_year, nl_sat, img_size, cardinality] ) + ".csv"

# table containing the long lat coordinates,corresponding nightlights and geocodes (eg. tambons) etc
# CEntroid and NIghttime data
CENI = "_".join( ["CNN", "CENI", country, img_size, cardinality] ) + ".csv"

# this is not a file that gets read in, but it is used to name single images. It is here for completeness sake. The example is image with the ID 123456
# Daytime IMaGe
DIMG = "_".join( ["CNN", "DIMG" , country, daytime_year, day_sat, img_res, img_size, "13"] )

# this is a zipped folder of all the images downloaded by a different script.
# IMaGe Bunch
IMGB     = "_".join( ["CNN", "IMGB" , country, daytime_year, day_sat, img_res, "JPG", img_size, number_bands, cardinality, "100PERC"] )
IMGB_ext = IMGB + ".tar.gz"

# here you have to add + ".pth" youself since the learn.load function of fastai adds it automatically, whereas other python code needs the extention
# Trained Convolutional Neural Network
TCNN = "_".join( ["CNN", "TCNN" , architecture, country, daytime_year, day_sat, nl_sat, img_res, img_size, number_bands, cardinality, validation_size] )
TCNN_ext = TCNN + ".pth"

# Poverty OUTput
# this is the output file that will be stored to disc
POUT = "_".join( ["CNN", "POUT" , architecture, country, prediction_year, day_sat, nl_sat,
                  img_res, img_size, number_bands, cardinality, sd_outliers, str(layer_drops), use_pop, validation_size] ) + ".csv"

# Trained RIDge regression
TRID = "_".join(["CNN", "TRID" , architecture, country, prediction_year, day_sat, nl_sat,
                 img_res, img_size, number_bands, cardinality, sd_outliers, str(layer_drops), use_pop, validation_size] )
TRID_ext = TRID + ".pkl"

# plots preamble
PLOT = "_".join(["CNN", "PLOT" , architecture, country, prediction_year, day_sat, nl_sat,
                 img_res, img_size, number_bands, cardinality, sd_outliers, str(layer_drops), use_pop, validation_size] )

# Cross validation outputs
CVOU = "_".join(["CNN", "CVOU" , architecture, country, prediction_year, day_sat, nl_sat,
                 img_res, img_size, number_bands, cardinality, sd_outliers, str(layer_drops), use_pop, validation_size] )

# Ridge Input data
FOUT = "_".join(["CNN", "FOUT" , architecture, country, prediction_year, day_sat, nl_sat,
                 img_res, img_size, number_bands, cardinality, sd_outliers, str(layer_drops), use_pop, validation_size] ) + ".csv"



print('CENT: ' + CENT)
print('NIGH: ' + NIGH)
print('CENT: ' + CENI)
print('DIMG: ' + DIMG)
print('IMGB: ' + IMGB)
print('TCNN: ' + TCNN)
print('POUT: ' + POUT)
print('TRID: ' + TRID)
print('CVOU: ' + CVOU)
print('FOUT: ' + FOUT)

CENT: CNN_CENT_COL_3840_ALL.csv
NIGH: CNN_NIGH_COL_2016_VI_3840_ALL.csv
CENT: CNN_CENI_COL_3840_ALL.csv
DIMG: CNN_DIMG_COL_2016_ST_384_3840_13
IMGB: CNN_IMGB_COL_2016_ST_384_JPG_3840_03_ALL_100PERC
TCNN: CNN_TCNN_RES34_COL_2016_ST_VI_384_3840_03_ALL_TST10
POUT: CNN_POUT_RES34_COL_2016_ST_VI_384_3840_03_ALL_04_2_usepopT_TST10.csv
TRID: CNN_TRID_RES34_COL_2016_ST_VI_384_3840_03_ALL_04_2_usepopT_TST10
CVOU: CNN_CVOU_RES34_COL_2016_ST_VI_384_3840_03_ALL_04_2_usepopT_TST10
FOUT: CNN_FOUT_RES34_COL_2016_ST_VI_384_3840_03_ALL_04_2_usepopT_TST10.csv

Various column names used througout the script.

In [0]:
target_variable_name = "MPM" #"pov_natdev_2018"
survey_area_name     = "geocode_" + survey_year
validation_col_name  = "validation_selection_" + survey_year
population_name      = "gridpop_" + survey_year

This selects the correct neural network specification based on the variables set in the beginning.

In [0]:
architecture_dict = {"RES34" : "resnet34", "RES50":"resnet50"}

try:
  architecture_dict[architecture]
except:
  print("architecture is not correctly specified, try RES32, RES50 or add to the architecture_dict")
  raise SystemExit(0)

Updates fastai

In [0]:
#!curl -s https://course.fast.ai/setup/colab | bash

sets up gdrive within fastai

We import all the necessary packages. We are going to work with the fastai V1 library which sits on top of Pytorch 1.0. The fastai library provides many useful functions that enable us to quickly and easily build neural networks and train our models.

In [0]:
from fastai.vision import *
from fastai.metrics import error_rate

Set up path for the CNN input data

In [0]:
root_col = '/root/.fastai/'

install secure connection library

In [0]:
!pip install 'pyOpenSSL>=0.11'

Requirement already satisfied: pyOpenSSL>=0.11 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (19.1.0)
Requirement already satisfied: six>=1.5.2 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from pyOpenSSL>=0.11) (1.14.0)
Requirement already satisfied: cryptography>=2.8 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from pyOpenSSL>=0.11) (2.8)
Requirement already satisfied: cffi!=1.11.3,>=1.8 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from cryptography>=2.8->pyOpenSSL>=0.11) (1.14.0)
Requirement already satisfied: pycparser in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from cffi!=1.11.3,>=1.8->cryptography>=2.8->pyOpenSSL>=0.11) (2.19)

sets up google cloud computing

In [0]:
from PIL import Image

CENI (centroids, nightlights, pop, image id, target var)
copy to VM and load into RAM

copy the zip file into the VM enviroment
If the file can't be found compare the output of the ls command with the name of the file in the error message. often a year or the satellite are wrongly specified, which can be changed in the first cell of this script or your paths might not be set up correctly.

In [0]:
df_full = pd.read_csv('/dbfs/mnt/pobreza/changeanalysis/func-mintic-pobrezamultidimensio/IMAGENES/CNN_CENI/cnn_ceni_col_' + survey_year + '_complete_JPG.csv')
df_full.head()


Out[61]:

,lat,long,id,name,bin_GMM
0,-74.078672,3.880722,40047,CNN_DIMG_COL_2016_ST_384_3840_13_040047.jpg,1
1,-74.044176,3.880722,40048,CNN_DIMG_COL_2016_ST_384_3840_13_040048.jpg,1
2,-74.009681,3.880722,40049,CNN_DIMG_COL_2016_ST_384_3840_13_040049.jpg,1
3,-73.871700,3.880722,40053,CNN_DIMG_COL_2016_ST_384_3840_13_040053.jpg,2
4,-73.837204,3.880722,40054,CNN_DIMG_COL_2016_ST_384_3840_13_040054.jpg,2


Check if the nightlight csv list of files corresponds 1:1 to the folder with the actual images
both outputs need to be empty objects 0 rows

In [0]:
all_img = !ls -1 /dbfs/mnt/pobreza/changeanalysis/func-mintic-pobrezamultidimensio/IMAGENES/2017/JPG
all_img = all_img.get_list()
all_img = pd.DataFrame(all_img)

missing_images_ID = df_full["name"].isin(all_img[0])
missing_csventry_ID = all_img[0].isin(df_full["bin_GMM"])

missing_images = df_full[~missing_images_ID]
missing_entries = all_img[~missing_csventry_ID]
print("images in the df_full, but not in the folder: ")
print(missing_images)

print("images in the folder, but not in the df_full: ")
print(missing_entries)


images in the df_full, but not in the folder: 
Empty DataFrame
Columns: [name, bin_GMM]
Index: []
images in the folder, but not in the df_full: 
 0
0 CNN_DIMG_COL_2016_ST_384_3840_13_000001.jpg
1 CNN_DIMG_COL_2016_ST_384_3840_13_000002.jpg
2 CNN_DIMG_COL_2016_ST_384_3840_13_000003.jpg
3 CNN_DIMG_COL_2016_ST_384_3840_13_000004.jpg
4 CNN_DIMG_COL_2016_ST_384_3840_13_000005.jpg
... ...
77974 CNN_DIMG_COL_2016_ST_384_3840_13_077975.jpg
77975 CNN_DIMG_COL_2016_ST_384_3840_13_077976.jpg
77976 CNN_DIMG_COL_2016_ST_384_3840_13_077977.jpg
77977 CNN_DIMG_COL_2016_ST_384_3840_13_077978.jpg
77978 Captura de Pantalla 2021-06-02 a la(s) 12.33.5...

[77979 rows x 1 columns]

In [0]:
df = df_full.copy(deep = True)[missing_images_ID]
print(df_full.shape)
print(df.shape)

(77978, 3)
(77978, 3)

Find the location of the label and filename column in the CENI / df file

In [0]:
label_column_name = "bin_GMM" 

label_column_id = df.columns.get_loc(label_column_name)
filename_column_id = df.columns.get_loc("name")

df.astype({label_column_name: 'int64'}).dtypes


print(df.columns[[label_column_id, filename_column_id]])

Index(['bin_GMM', 'name'], dtype='object')

load dataset into  ImageDataBunch.

fn_col specifies the column in with the image paths are stored.

label_col specifies the column in which the corresponding labels are (in this case nightlight intensity)

In [0]:
from fastai import *
from pathlib import Path
import os

root_col='/dbfs/mnt/pobreza/changeanalysis/func-mintic-pobrezamultidimensio/IMAGENES/'
data = ImageDataBunch.from_df(
path = root_col,
df = df,
folder = survey_year + '/' + 'JPG',
valid_pct = 0.2,
fn_col = filename_column_id, 
label_col = label_column_id )

/databricks/python/lib/python3.7/site-packages/fastai/basic_data.py:272: UserWarning: It's not possible to collate samples of your dataset together in a batch.
Shapes of the inputs/targets:
[[torch.Size([3, 386, 385]), torch.Size([3, 386, 389]), torch.Size([3, 386, 384]), torch.Size([3, 386, 387]), torch.Size([3, 386, 385]), torch.Size([3, 386, 383]), torch.Size([3, 386, 383]), torch.Size([3, 386, 385]), torch.Size([3, 386, 383]), torch.Size([3, 386, 385]), torch.Size([3, 386, 385]), torch.Size([3, 386, 385]), torch.Size([3, 386, 391]), torch.Size([3, 386, 383]), torch.Size([3, 386, 383]), torch.Size([3, 386, 387]), torch.Size([3, 386, 387]), torch.Size([3, 386, 384]), torch.Size([3, 386, 385]), torch.Size([3, 386, 383]), torch.Size([3, 386, 383]), torch.Size([3, 386, 384]), torch.Size([3, 386, 384]), torch.Size([3, 386, 384]), torch.Size([3, 386, 390]), torch.Size([3, 386, 383]), torch.Size([3, 386, 389]), torch.Size([3, 386, 391]), torch.Size([3, 386, 388]), torch.Size([3, 386, 383]), torch.Size([3, 386, 383]), torch.Size([3, 386, 385]), torch.Size([3, 386, 383]), torch.Size([3, 386, 384]), torch.Size([3, 386, 385]), torch.Size([3, 386, 383]), torch.Size([3, 386, 383]), torch.Size([3, 386, 387]), torch.Size([3, 386, 388]), torch.Size([3, 386, 384]), torch.Size([3, 386, 383]), torch.Size([3, 386, 385]), torch.Size([3, 386, 386]), torch.Size([3, 386, 384]), torch.Size([3, 386, 387]), torch.Size([3, 386, 386]), torch.Size([3, 386, 383]), torch.Size([3, 386, 384]), torch.Size([3, 386, 384]), torch.Size([3, 386, 384]), torch.Size([3, 386, 383]), torch.Size([3, 386, 386]), torch.Size([3, 386, 383]), torch.Size([3, 386, 385]), torch.Size([3, 386, 383]), torch.Size([3, 386, 383]), torch.Size([3, 386, 387]), torch.Size([3, 386, 383]), torch.Size([3, 386, 383]), torch.Size([3, 386, 384]), torch.Size([3, 386, 387]), torch.Size([3, 386, 383]), torch.Size([3, 386, 383]), torch.Size([3, 386, 386])], [(), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), ()]]
 warn(message)
You can deactivate this warning by passing `no_check=True`.

In [0]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable

We initialize a cnn learning object from the fastai library.
 This binds together the data (lables and images) in a neat package. 
  We won't do any actual CNN learning in this script since we load a network that has already been trained to our purpose, but the object is still called learner
 


 'pretrained = False' for that very reason. If it was True this function would download a network that has been pretrained on imagenet images, but since we use a network that has been trained by us we can skip it and start with random weights here.

In [0]:
toeval = "cnn_learner(data, models." + architecture_dict[architecture] + ", metrics=error_rate, pretrained = False)" # eval a string is just a trick to use the naming convention automatically
learn = eval(toeval)

del toeval

load Toms pretrained model. In this step we overwrite the model parameters of the object we just loaded (called learner). This is why we can load an empty/random model - we overwrite the parameters with an existing model

In [0]:

# load it to python
learn.load(root_col + 'models/Ex1_COL_' + survey_year + '_ST_VI_384_3840_13_ALL_trn90_final' )

In [0]:
path_one = df["name"][1000]
path_two = df["name"][50]

pic_one = open_image(root_col + survey_year + "/JPG" +"/" + path_one)
pic_two = open_image(root_col + survey_year + "/JPG" +"/" + path_two)

Here we insert our own predict function as a method to the Learner class. 
this method only returns the node values of the last layer in the model. 
normally these are the probabilities of each output category. 
since the predict function also returns the correspoding label with the max probability we cannot use it (we have a lot more output nodes than labels in the truncated model). Once we truncate the model and our output vector is a lot longer than we have labels.

In [0]:
def my_predict(self, item:ItemBase, return_x:bool=False, batch_first:bool=True, with_dropout:bool=False, **kwargs):
      "Return probabilities for `item`."
      batch = self.data.one_item(item)
      res = self.pred_batch(batch=batch, with_dropout=with_dropout)
      raw_pred,x = grab_idx(res,0,batch_first=batch_first),batch[0]
      return (raw_pred)


setattr(Learner, 'my_predict', my_predict)

take a test image and try it on our new method
We can see that our function only

In [0]:
print(learn.predict (pic_one))
print(learn.my_predict (pic_one))

--------------------------------------------------------------------------- 
 IndexError Traceback (most recent call last)
 <command-91444125656597> in <module> 
 ----> 1 print ( learn . predict ( pic_one ) ) 
 2 print ( learn . my_predict ( pic_one ) ) 

 /databricks/python/lib/python3.7/site-packages/fastai/basic_train.py in predict (self, item, return_x, batch_first, with_dropout, **kwargs) 
 380 pred = ds . y . analyze_pred ( raw_pred , ** kwargs ) 
 381 x = ds . x . reconstruct ( grab_idx ( x , 0 ) ) 
 --> 382 y = ds . y . reconstruct ( pred , x ) if has_arg ( ds . y . reconstruct , 'x' ) else ds . y . reconstruct ( pred ) 
 383 return ( x , y , pred , raw_pred ) if return_x else ( y , pred , raw_pred ) 
 384 

 /databricks/python/lib/python3.7/site-packages/fastai/data_block.py in reconstruct (self, t) 
 388 
 389 def reconstruct ( self , t ) : 
 --> 390 return Category ( t , self . classes [ t ] ) 
 391 
 392 class MultiCategoryProcessor ( CategoryProcessor ) : 

 IndexError : list index out of range

remove last fully-connected layer

In [0]:
new_model = learn


print('learn length: '     + str(len(learn.model[1])))
print('new_model length: ' + str(len(new_model.model[1])))

new_model.model[1] = new_model.model[1][:-layer_drops]


print('learn length: '     + str(len(learn.model[1])))
print('new_model length: ' + str(len(new_model.model[1])))

learn length: 9
new_model length: 9
learn length: 7
new_model length: 7

JPGS are 3 dimensional, but pytorch wants 4 dimensions where the first is the sample size.
So to scale the image, turn it to a tensor cuda object (cuda means it is suitable to be used by a GPU) and normalize the values. 

The images need to be converted to a tensor, scaled to the correct size and normalized.

In [0]:
def Extract_Features (img):
    # Input:  A jpg with the correct size (we could resize but this usually happens before in our case)
    # return: A tensor of image features (second to last layer of the CNN)
    # those are the features that get weighted together to classify the image's nightlight class.

    img_feature_layer = new_model.my_predict(img).flatten()
    
    return(img_feature_layer)

In [0]:
tensor_len = len(Extract_Features (pic_one))
print(tensor_len)

512

Now "predict" the image features.
First we create an output dataframe

In [0]:
features_out = np.empty((len(df["id"]), tensor_len))
features_out_img = df["name"]
# pred_out = np.empty((len(df["filename"]),1))
print(features_out.shape)
# print(pred_out.shape)
dfsave = df.copy(deep = True)

(77978, 512)

In [0]:
for i, path_i in enumerate(features_out_img):
  # open the image with the fastai open image function
  temp_img = open_image(root_col + survey_year + "/JPG" + "/"  + path_i)
  # extract the features of the single image
  tempfeatures = Extract_Features (temp_img).flatten().reshape(1, -1).numpy()
  # store them for output
  features_out[i,:] = tempfeatures
  # get fitted values of the ridge regression and save the them 
#   pred_out[i]       = float(best_ridge.predict(tempfeatures))


print(dfsave.shape)
print(df.shape)

(77978, 3)
(77978, 3)

Merging the output features with the vector of image ids

In [0]:
features_out_pd = pd.DataFrame(data = features_out, index = features_out_img)

In [0]:
features_out_pd.head()

Out[58]:

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
CNN_DIMG_COL_2016_ST_384_3840_13_040047.jpg,0.002375,0.002076,0.001997,0.002059,0.003238,0.002092,0.001511,0.001786,0.001866,0.002170,0.001913,0.001954,0.001749,0.001901,0.002731,0.001723,0.001887,0.002184,0.002167,0.001956,0.002375,0.001993,0.002016,0.001924,0.001661,0.001941,0.002295,0.001830,0.002029,0.001667,0.001833,0.001788,0.001931,0.001834,0.001935,0.003179,0.001939,0.001421,0.002024,0.002040,...,0.002040,0.001875,0.001544,0.001796,0.001977,0.002072,0.001632,0.002100,0.002059,0.001971,0.002100,0.001785,0.001924,0.001967,0.001634,0.001964,0.001895,0.002040,0.001751,0.002052,0.002122,0.001699,0.001875,0.002008,0.001813,0.001606,0.001807,0.001803,0.001816,0.002524,0.001506,0.001839,0.001976,0.001940,0.002386,0.002039,0.001930,0.001838,0.001669,0.001486
CNN_DIMG_COL_2016_ST_384_3840_13_040048.jpg,0.002344,0.002053,0.002009,0.001986,0.003129,0.002053,0.001621,0.001796,0.001877,0.002182,0.001924,0.001972,0.001759,0.001912,0.002710,0.001765,0.001898,0.002197,0.002180,0.001948,0.002290,0.002004,0.002028,0.001928,0.001677,0.001953,0.002301,0.001840,0.002041,0.001677,0.001844,0.001799,0.001942,0.001845,0.001947,0.002897,0.001950,0.001429,0.002035,0.002084,...,0.002009,0.001886,0.001553,0.001807,0.001989,0.002085,0.001642,0.002040,0.002071,0.001982,0.002112,0.001796,0.001936,0.001979,0.001643,0.001975,0.001907,0.002052,0.001773,0.002064,0.002134,0.001709,0.001886,0.002020,0.001823,0.001615,0.001818,0.001761,0.001792,0.002299,0.001498,0.001850,0.001988,0.001951,0.002265,0.002050,0.001941,0.001849,0.001679,0.001535
CNN_DIMG_COL_2016_ST_384_3840_13_040049.jpg,0.002743,0.002083,0.001981,0.002002,0.003346,0.002021,0.001528,0.001771,0.001850,0.002165,0.001897,0.001938,0.001734,0.001885,0.002784,0.001754,0.001871,0.002166,0.002150,0.001924,0.002354,0.001976,0.002006,0.001937,0.001692,0.001925,0.002409,0.001815,0.002013,0.001653,0.001818,0.001774,0.001915,0.001819,0.001919,0.003009,0.001923,0.001409,0.002007,0.002053,...,0.002080,0.001859,0.001531,0.001781,0.001961,0.002055,0.001619,0.002070,0.002042,0.001955,0.002082,0.001771,0.001908,0.001951,0.001620,0.001947,0.001881,0.002023,0.001758,0.002035,0.002104,0.001685,0.001860,0.001991,0.001798,0.001593,0.001792,0.001704,0.001789,0.002577,0.001453,0.001824,0.001960,0.001924,0.002331,0.002022,0.001914,0.001823,0.001655,0.001484
CNN_DIMG_COL_2016_ST_384_3840_13_040053.jpg,0.002679,0.002089,0.001987,0.002026,0.003221,0.002033,0.001581,0.001776,0.001856,0.002179,0.001903,0.001944,0.001739,0.001891,0.002719,0.001753,0.001877,0.002173,0.002156,0.001929,0.002291,0.001982,0.002010,0.001925,0.001690,0.001931,0.002384,0.001820,0.002019,0.001658,0.001823,0.001779,0.001921,0.001825,0.001925,0.002928,0.001929,0.001414,0.002013,0.002060,...,0.002055,0.001865,0.001536,0.001787,0.001967,0.002062,0.001624,0.002083,0.002049,0.001961,0.002089,0.001776,0.001914,0.001957,0.001625,0.001953,0.001887,0.002029,0.001774,0.002041,0.002111,0.001690,0.001866,0.001997,0.001803,0.001597,0.001798,0.001716,0.001793,0.002553,0.001463,0.001829,0.001966,0.001930,0.002314,0.002028,0.001920,0.001829,0.001660,0.001496
CNN_DIMG_COL_2016_ST_384_3840_13_040054.jpg,0.002820,0.002047,0.001986,0.001927,0.003220,0.001930,0.001791,0.001776,0.001855,0.002173,0.001902,0.001950,0.001739,0.001890,0.002718,0.001766,0.001876,0.002172,0.002155,0.001923,0.002228,0.001982,0.002012,0.001921,0.001692,0.001931,0.002413,0.001820,0.002018,0.001658,0.001823,0.001779,0.001920,0.001824,0.001925,0.002768,0.001928,0.001413,0.002012,0.002074,...,0.002058,0.001864,0.001535,0.001786,0.001966,0.002061,0.001623,0.001995,0.002048,0.001960,0.002088,0.001776,0.001914,0.001957,0.001625,0.001953,0.001887,0.0020

In [0]:
features_out = pd.merge(df_full,features_out_pd,on='name',how='left')

In [0]:
features_out.head()

Out[63]:

,lat,long,id,name,bin_GMM,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,-74.078672,3.880722,40047,CNN_DIMG_COL_2016_ST_384_3840_13_040047.jpg,1,0.002375,0.002076,0.001997,0.002059,0.003238,0.002092,0.001511,0.001786,0.001866,0.002170,0.001913,0.001954,0.001749,0.001901,0.002731,0.001723,0.001887,0.002184,0.002167,0.001956,0.002375,0.001993,0.002016,0.001924,0.001661,0.001941,0.002295,0.001830,0.002029,0.001667,0.001833,0.001788,0.001931,0.001834,0.001935,...,0.002040,0.001875,0.001544,0.001796,0.001977,0.002072,0.001632,0.002100,0.002059,0.001971,0.002100,0.001785,0.001924,0.001967,0.001634,0.001964,0.001895,0.002040,0.001751,0.002052,0.002122,0.001699,0.001875,0.002008,0.001813,0.001606,0.001807,0.001803,0.001816,0.002524,0.001506,0.001839,0.001976,0.001940,0.002386,0.002039,0.001930,0.001838,0.001669,0.001486
1,-74.044176,3.880722,40048,CNN_DIMG_COL_2016_ST_384_3840_13_040048.jpg,1,0.002344,0.002053,0.002009,0.001986,0.003129,0.002053,0.001621,0.001796,0.001877,0.002182,0.001924,0.001972,0.001759,0.001912,0.002710,0.001765,0.001898,0.002197,0.002180,0.001948,0.002290,0.002004,0.002028,0.001928,0.001677,0.001953,0.002301,0.001840,0.002041,0.001677,0.001844,0.001799,0.001942,0.001845,0.001947,...,0.002009,0.001886,0.001553,0.001807,0.001989,0.002085,0.001642,0.002040,0.002071,0.001982,0.002112,0.001796,0.001936,0.001979,0.001643,0.001975,0.001907,0.002052,0.001773,0.002064,0.002134,0.001709,0.001886,0.002020,0.001823,0.001615,0.001818,0.001761,0.001792,0.002299,0.001498,0.001850,0.001988,0.001951,0.002265,0.002050,0.001941,0.001849,0.001679,0.001535
2,-74.009681,3.880722,40049,CNN_DIMG_COL_2016_ST_384_3840_13_040049.jpg,1,0.002743,0.002083,0.001981,0.002002,0.003346,0.002021,0.001528,0.001771,0.001850,0.002165,0.001897,0.001938,0.001734,0.001885,0.002784,0.001754,0.001871,0.002166,0.002150,0.001924,0.002354,0.001976,0.002006,0.001937,0.001692,0.001925,0.002409,0.001815,0.002013,0.001653,0.001818,0.001774,0.001915,0.001819,0.001919,...,0.002080,0.001859,0.001531,0.001781,0.001961,0.002055,0.001619,0.002070,0.002042,0.001955,0.002082,0.001771,0.001908,0.001951,0.001620,0.001947,0.001881,0.002023,0.001758,0.002035,0.002104,0.001685,0.001860,0.001991,0.001798,0.001593,0.001792,0.001704,0.001789,0.002577,0.001453,0.001824,0.001960,0.001924,0.002331,0.002022,0.001914,0.001823,0.001655,0.001484
3,-73.871700,3.880722,40053,CNN_DIMG_COL_2016_ST_384_3840_13_040053.jpg,2,0.002679,0.002089,0.001987,0.002026,0.003221,0.002033,0.001581,0.001776,0.001856,0.002179,0.001903,0.001944,0.001739,0.001891,0.002719,0.001753,0.001877,0.002173,0.002156,0.001929,0.002291,0.001982,0.002010,0.001925,0.001690,0.001931,0.002384,0.001820,0.002019,0.001658,0.001823,0.001779,0.001921,0.001825,0.001925,...,0.002055,0.001865,0.001536,0.001787,0.001967,0.002062,0.001624,0.002083,0.002049,0.001961,0.002089,0.001776,0.001914,0.001957,0.001625,0.001953,0.001887,0.002029,0.001774,0.002041,0.002111,0.001690,0.001866,0.001997,0.001803,0.001597,0.001798,0.001716,0.001793,0.002553,0.001463,0.001829,0.001966,0.001930,0.002314,0.002028,0.001920,0.001829,0.001660,0.001496
4,-73.837204,3.880722,40054,CNN_DIMG_COL_2016_ST_384_3840_13_040054.jpg,2,0.002820,0.002047,0.001986,0.001927,0.003220,0.001930,0.001791,0.001776,0.001855,0.002173,0.001902,0.001950,0.001739,0.001890,0.002718,0.001766,0.001876,0.002172,0.002155,0.001923,0.002228,0.001982,0.002012,0.001921,0.001692,0.001931,0.002413,0.001820,0.002018,0.001658,0.001823,0.001779,0.001920,0.001824,0.001925,...,0.002058,0.001864,0.001535,0.001786,0.001966,0.002061,0.001623,0.001995,0.002048,0.001960,0.002088,0.001776,0.001914,0.001957,0.001625,0.001953,0.001887,0.002029,0.001746,0.002040,0.002110,0.001690,0.001865,0.001997,0.001803,0.001597,0.001797,0.001647,0.001772,0.002507,0.001418,0.001829,0.001965,0.001929,0.00

Save the image level features to disc

In [0]:
# save to disc
features_out.to_csv(root_col + 'FEATURES/' + 'CNN_FOUT_RES34_COL_' + survey_year + '_ST_VI_384_3840_03_ALL_04.csv')
